In [11]:
import pandas as pd
import numpy as np
import scipy.stats

from sklearn.metrics.pairwise import cosine_similarity

In [12]:
# movies = pd.read_csv('movies.csv')
# ratings = pd.read_csv('ratings.csv')

# print(movies.head())
# print(ratings.head())

In [13]:
# df = pd.merge(ratings, movies, on= 'movieId', how= 'inner')

# df.head()

In [14]:
movie_ratings = pd.read_csv('combined_movie_data_all_genres_in_one_column.csv')

In [15]:
df = pd.DataFrame(movie_ratings)
df.head()

,userid,user_name,rating,movieid,title,released_year,genre
0,1,Liam,1.0,3176,The Talented Mr. Ripley,1999.0,Drama
1,1,Liam,1.0,3176,The Talented Mr. Ripley,1999.0,Mystery
2,1,Liam,1.0,3176,The Talented Mr. Ripley,1999.0,Thriller
3,1,Liam,2.0,2338,I Still Know What You Did Last Summer,1998.0,Horror
4,1,Liam,2.0,2338,I Still Know What You Did Last Summer,1998.0,Mystery


In [16]:
aggrate = df.groupby('title').agg(meanrate = ('rating', 'mean'), numberrating = ('rating', 'count')).reset_index()

In [17]:
aggrate200 = aggrate[aggrate['numberrating'] > 200]

aggrate200.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294 entries, 66 to 9318
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         294 non-null    object 
 1   meanrate      294 non-null    float64
 2   numberrating  294 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 9.2+ KB


In [18]:
aggrate200.sort_values(by= 'numberrating', ascending= False).head()

,title,meanrate,numberrating
2653,Forrest Gump,4.164134,1316
5531,Pulp Fiction,4.197068,1228
8743,Toy Story,3.920930,1075
7737,The Lion King,3.941860,1032
6152,Shrek,3.867647,1020


In [19]:
df200 = pd.merge(df, aggrate200[['title']], on= 'title', how= 'inner')

df200.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106233 entries, 0 to 106232
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   userid         106233 non-null  int64  
 1   user_name      106233 non-null  object 
 2   rating         106233 non-null  float64
 3   movieid        106233 non-null  int64  
 4   title          106233 non-null  object 
 5   released_year  106233 non-null  float64
 6   genre          106233 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 5.7+ MB


In [22]:
matrix = df200.pivot_table(index= 'userid', columns= 'title', values= 'rating')

matrix.head()

title,2001: A Space Odyssey,300,A Beautiful Mind,A Bug's Life,A Clockwork Orange,Addams Family Values,Aladdin,Alice in Wonderland,Alien,Aliens,...,Wallace & Gromit: The Wrong Trousers,War of the Worlds,Watchmen,Waterworld,Who Framed Roger Rabbit?,Wild Wild West,Willy Wonka & the Chocolate Factory,X-Men,X-Men: First Class,X2: X-Men United
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,4.0,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,5.0,5.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
maxtrixnorm = matrix.subtract(matrix.mean(axis= 1), axis = 'rows')

maxtrixnorm.head()

title,2001: A Space Odyssey,300,A Beautiful Mind,A Bug's Life,A Clockwork Orange,Addams Family Values,Aladdin,Alice in Wonderland,Alien,Aliens,...,Wallace & Gromit: The Wrong Trousers,War of the Worlds,Watchmen,Waterworld,Who Framed Roger Rabbit?,Wild Wild West,Willy Wonka & the Chocolate Factory,X-Men,X-Men: First Class,X2: X-Men United
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,0.590909,NaN,NaN,0.590909,-0.409091,NaN,...,NaN,NaN,NaN,NaN,0.590909,NaN,0.590909,0.590909,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,0.660000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.660000,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,-0.482759,0.517241,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
simuser = maxtrixnorm.T.corr()
simuser.head()

userid,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userid,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,NaN,0.350672,0.275476,-0.545115,-0.109770,0.582451,1.0,-0.037987,...,0.091574,0.125294,0.037327,-0.522233,-0.150790,0.282179,0.103400,0.122956,-0.516398,0.028902
2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.500000,...,-0.292509,NaN,-1.000000,NaN,NaN,0.583333,NaN,-0.216295,NaN,0.686764
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.350672,NaN,NaN,1.000000,-0.493399,0.244493,0.722188,0.419785,NaN,0.302486,...,-0.319149,0.112687,0.156200,-0.158114,0.512206,-0.066847,0.102240,-0.298252,NaN,-0.164371
5,0.275476,NaN,NaN,-0.493399,1.000000,-0.021256,0.328889,0.062284,NaN,-0.777714,...,0.000000,0.165380,0.172046,-0.021179,-0.364156,0.332960,0.184466,0.017970,0.365148,-0.059170


In [34]:
simuserco = cosine_similarity(maxtrixnorm.fillna(0))
simuserco


array([[ 1.        ,  0.00256394,  0.        , ...,  0.06776372,
        -0.08892368,  0.01896628],
       [ 0.00256394,  1.        ,  0.        , ..., -0.01852975,
        -0.22068371,  0.11708842],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.06776372, -0.01852975,  0.        , ...,  1.        ,
         0.11213097,  0.01594316],
       [-0.08892368, -0.22068371,  0.        , ...,  0.11213097,
         1.        , -0.10391163],
       [ 0.01896628,  0.11708842,  0.        , ...,  0.01594316,
        -0.10391163,  1.        ]])

In [37]:
pickeduser = 1

# Remove picked user ID from the candidate list
simuserco = np.delete(simuserco, pickeduser, axis=0)

# Take a look at the data
print(simuserco)

[[ 1.          0.00256394  0.         ...  0.06776372 -0.08892368
   0.01896628]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.15391664 -0.08764429  0.         ... -0.11225134 -0.08273935
  -0.02473525]
 ...
 [ 0.06776372 -0.01852975  0.         ...  1.          0.11213097
   0.01594316]
 [-0.08892368 -0.22068371  0.         ...  0.11213097  1.
  -0.10391163]
 [ 0.01896628  0.11708842  0.         ...  0.01594316 -0.10391163
   1.        ]]


In [38]:
numuser = 10


In [39]:
usersimthreshold = .3


In [44]:
similaruser = simuser[simuser[pickeduser] > usersimthreshold][pickeduser].sort_values(ascending= False)[:numuser]

# similar_users = user_similarity[user_similarity[picked_userid]>user_similarity_threshold][picked_userid].sort_values(ascending=False)[:n]

print(f'The similar users for user {pickeduser} are', similaruser)

The similar users for user 1 are userid
1      1.000000
106    1.000000
598    1.000000
9      1.000000
550    1.000000
108    1.000000
535    0.968496
431    0.944911
511    0.879883
236    0.875000
Name: 1, dtype: float64


In [45]:
pickeduserseen = maxtrixnorm[maxtrixnorm.index == pickeduser].dropna(axis= 1, how= 'all')
pickeduserseen

title,A Clockwork Orange,Alice in Wonderland,Alien,American Beauty,American History X,Apocalypse Now,Austin Powers: International Man of Mystery,Back to the Future,Back to the Future Part III,Batman,...,The Terminator,The Three Musketeers,The Usual Suspects,The Wizard of Oz,Total Recall,Toy Story,Twister,Who Framed Roger Rabbit?,Willy Wonka & the Chocolate Factory,X-Men
userid,,,,,,,,,,,,,,,,,,,,,
1,0.590909,0.590909,-0.409091,0.590909,0.590909,-0.409091,0.590909,0.590909,-0.409091,-0.409091,...,0.590909,-0.409091,0.590909,0.590909,-0.409091,-0.409091,-1.409091,0.590909,0.590909,0.590909


In [46]:
similarmovies = maxtrixnorm[maxtrixnorm.index.isin(similaruser.index)].dropna(axis= 1, how= 'all')
similarmovies

title,A Beautiful Mind,A Clockwork Orange,Alice in Wonderland,Alien,"Amelie (Fabuleux destin d'Amélie Poulain, Le)",American Beauty,American History X,American Pie,Apocalypse Now,Armageddon,...,Titanic,Total Recall,Toy Story,Twister,Up,Vertigo,WALL·E,Who Framed Roger Rabbit?,Willy Wonka & the Chocolate Factory,X-Men
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.590909,0.590909,-0.409091,NaN,0.590909,0.590909,NaN,-0.409091,NaN,...,NaN,-0.409091,-0.409091,-1.409091,NaN,NaN,NaN,0.590909,0.590909,0.590909
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,0.75,NaN,NaN,NaN,0.750,NaN,NaN,NaN,NaN,-2.25,...,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,NaN,NaN,NaN,0.166667,NaN,0.166667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
511,NaN,NaN,NaN,NaN,-0.475,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.025000,NaN,NaN,NaN,NaN,NaN
535,NaN,NaN,NaN,NaN,NaN,1.428571,NaN,0.428571,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.336538,NaN,0.163462,NaN,-0.336538,NaN,NaN,NaN


In [47]:
similarmovies.drop(pickeduserseen.columns, axis= 1, inplace= True, errors= 'ignore')

In [48]:
similarmovies

title,A Beautiful Mind,"Amelie (Fabuleux destin d'Amélie Poulain, Le)",American Pie,Armageddon,Austin Powers: The Spy Who Shagged Me,Avatar,Back to the Future Part II,Blade Runner,Catch Me If You Can,Chicken Run,...,The Hunger Games,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Two Towers,The Shawshank Redemption,The Sixth Sense,Titanic,Up,Vertigo,WALL·E
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,-1.222222,NaN,NaN,NaN,...,NaN,0.777778,NaN,0.777778,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,...,NaN,0.250000,0.250,0.250000,0.250000,NaN,NaN,NaN,NaN,NaN
108,0.75,0.750,NaN,-2.25,NaN,NaN,NaN,0.75,0.750000,-1.25,...,NaN,-1.250000,NaN,-1.250000,NaN,NaN,-0.25,NaN,NaN,NaN
236,NaN,NaN,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
511,NaN,-0.475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.975,0.525000,0.525,NaN,0.525000,NaN,NaN,0.025000,NaN,NaN
535,NaN,NaN,0.428571,NaN,-2.571429,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.428571,NaN,NaN,NaN,NaN
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.336538,NaN,...,NaN,NaN,NaN,NaN,0.163462,NaN,NaN,0.163462,NaN,-0.336538


In [49]:
# A dictionary to store item scores
item_score = {}

# Loop through items
for i in similarmovies.columns:
  # Get the ratings for movie i
  movie_rating = similarmovies[i]
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in similaruser.index:
    # If the movie has rating
    if pd.isna(movie_rating[u]) == False:
      # Score is the sum of user similarity score multiply by the movie rating
      score = similaruser[u] * movie_rating[u]
      # Add the score to the total score for the movie so far
      total += score
      # Add 1 to the count
      count +=1
  # Get the average score for the item
  item_score[i] = total / count

# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
    
# Sort the movies by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

# Select top m movies
m = 10
ranked_item_score.head(m)

,movie,movie_score
29,Little Miss Sunshine,1.692308
19,Harry Potter and the Order of the Phoenix,1.692308
15,Harry Potter and the Chamber of Secrets,1.692308
13,Eternal Sunshine of the Spotless Mind,1.692308
53,The Sixth Sense,1.383566
23,Inception,0.939236
12,Donnie Darko,0.750000
32,"O Brother, Where Art Thou?",0.750000
0,A Beautiful Mind,0.750000
7,Blade Runner,0.750000
